In [ ]:
!pip install openai weave wandb pandas numpy matplotlib seaborn python-docx nbformat
import os
import re
import weave
import wandb
import pandas as pd
import numpy as np
from openai import OpenAI
from typing import Dict, List, Any
import json
from datetime import datetime

# 初始化 Weave 和 W&B
weave.init('oral-history-rewriting-experiment-v3')

# DeepSeek 客户端配置
client = OpenAI(
    api_key=os.environ.get('DEEPSEEK_API_KEY'),
    base_url="https://api.deepseek.com/v1"
)

# 改进的提示词模板定义
BASE_PROMPT = """
你是一个专业的口述历史整理者。请将以下老年人口述采访内容改写成流畅的叙述性文字。

**重要要求：**
1. 保持原文的所有重要细节和具体内容，不要过度概括
2. 保留原文的情感和个人化表达
3. 字数不少于原文的20%
4. 按照时间顺序和主题进行组织，但不要丢失具体事例

**现在请改写以下内容：**
{oral_text}
"""

FEW_SHOT_PROMPT = """
你是一个专业的口述历史整理者。请参考以下示例，将新的口述内容改写成流畅的叙述性文字。注意示例仅供参考。

**示例改写：**
坑塘,就是散落于村庄内外的水塘,北方农村口语称它们为“坑”。梁庄有大大小小六个坑塘。小学前边有一个大坑塘,中间有一条窄窄、弯曲的小路把它隔开,这是童年时代我们上小学的必经之路。一到夏天,暴雨过后,这条路便成为一条“魔鬼小道”,坑塘的水往往会漫过小路,只留下断断续续的残面。几个小伙伴手拉手,打着赤脚,走着走着,就会听见“扑通”落水的声音。好在坑塘边的坡很缓,水也并不深,都能顺利爬上来。如果遇到连绵的阴雨天气,那就糟糕了,村里到处都是泥疗,猪粪、鸡屎被泡得到处流,一些碎石头、碎砖块不知从哪儿钻出来,一不小心就得脚生疼。从家里到学校不过三百米的路,我们一路上不知要踩到多少粪便,看着脚趾缝里挤出来黑色或黄色的污物,散发着臭味的粪便,那浑身的汗毛都要竖起来。虽然如此,小学前的坑塘仍然留有我美好的回忆。坑塘里种满了莲藕,一到夏天,青青的荷叶铺满整个坑塘,间或有粉红色的花高高地冒出来,随风摇曳。然后,慢慢变成莲蓬,里面的莲子圆圆的,鼓鼓的。等不及到成熟的时候,趁大人不注意,我们几个小伙伴会手拉手,连成一串, 进水里,去摘那最近的莲蓬。那莲子,咬一口,满嘴的清香。还有就是那个有着青石桥的坑塘。青石桥把一个大的坑塘分为两个,左边坑塘进入到村庄里面,右边坑塘往外延伸到公路旁,旁边有一条较宽的土路,也从村子的另一边通向公路。土路往上,就是梁家的自留地,每家约有几分地,种些辣椒、茄子、萝卜等蔬菜自给。路和自留地中间有一棵野生的大桑葚树。每到春末夏初,紫红的桑葚结满一树,女生用土块、棍子打,桑葚落了下来,砸进土里,浸满了灰尘,根本无法吃。那些男孩儿却“蹭蹭”地爬上去,摘满一兜,一溜烟就跑。

**示例改写：**
整整下了两天的雨。雨水洗刷下的原野清新、干净,树叶、庄稼都绿得发亮,灰暗的天空形成了一个封闭、安静而又辽阔的世界。而我们就在这时大时小的雨雾中穿行。世界那么小又那么无穷无尽,我们像在孤独中漫游,又像在无限神秘的大地中探索。公路两旁的水沟在多年的干涸之后终于又获得了新生,河流翻出了巨大的波浪和旋涡,在树林间游戏。我仿佛又回到了那湿润的童年,雨后在小沟渠捉鱼,踩水的欢乐和对自然无邪的亲近,每个人在此时都是最纯洁的。雨季来了。虽然不是南方,但每年的这个季节总会有十几天在连续下雨。我喜欢这样的雨天,雨“哗哗”下着,但并不阴暗,灰色的、发亮的天空辽阔、肃穆,给人一种庄严与阔大之感。河坡的树林是近几年才栽种的,林间还没有长出足够覆盖地面的草。赤脚踩在沙土路上,细细的、湿湿的砂石轻 人的脚,微疼微痒,感觉非常舒服。河水“哗哗”奔腾而去,充满力量和向往,那巨大的芦苇丛接受着雨水的冲刷,稳重而又充满生命力。雨中的河,升腾着雾气,苍茫无边,却又具有永恒的清新。河坡地里散落着许多小屋,基本上都是人们为看守庄稼而建的,在一片片空阔的沙地上,种了许多西瓜和花生,它们最适宜在沙地上种植。偶尔可以看到一两个身影在西瓜地里忙碌,估计是在检查西瓜的情况。这样的连阴雨对种瓜的人来说,是非常不好的事情。我们在一个开着门的小屋前张望,里面有一位妇女正在做家务,旁边一个三四岁的女孩子在玩耍。听到我们的声音,那位妇女扭过身来。哥哥笑了起来,这不是焕嫂子吗?

**重要要求：**
1. 保持原文的所有重要细节和具体内容，不要过度概括
2. 保留原文的情感和个人化表达
3. 字数不少于原文的20%
4. 按照时间顺序和主题进行组织，但不要丢失具体事例

**现在请改写以下内容：**
{oral_text}
"""

COT_PROMPT = """
你是一个专业的口述历史整理者。请按照以下步骤分析和改写口述内容，确保保留所有重要细节：

**第一步：详细提取时间线和关键事件**
- 识别所有时间线索和具体事件
- 建立完整的时间顺序
- 标记每个重要事件的具体细节

**第二步：分析人物关系和互动**
- 识别所有提到的人物及其关系
- 记录人物之间的具体互动和对话
- 分析人物情感变化

**第三步：梳理事件逻辑和因果关系**
- 提取所有主要和次要事件
- 分析事件之间的因果关系
- 保留事件的具体情境和背景

**第四步：理解情感基调和细节**
- 分析原始情感表达的细微差别
- 保持情感真实性，不美化负面情绪
- 保留情感表达的具体触发点

**第五步：基于以上详细分析进行改写，确保：**
- 保留原文的具体细节
- 字数不少于原文的20%
- 维持原文的个人化表达风格

**重要要求：**
1. 保持原文的所有重要细节和具体内容，不要过度概括
2. 保留原文的情感和个人化表达
3. 字数不少于原文的20%
4. 按照时间顺序和主题进行组织，但不要丢失具体事例

**现在请改写以下内容：**
{oral_text}
"""

COMBINED_PROMPT = """
你是一个专业的口述历史整理者。请参考示例并按照思维链步骤分析和改写口述内容，确保保留所有重要细节：

**示例改写：**
坑塘,就是散落于村庄内外的水塘,北方农村口语称它们为“坑”。梁庄有大大小小六个坑塘。小学前边有一个大坑塘,中间有一条窄窄、弯曲的小路把它隔开,这是童年时代我们上小学的必经之路。一到夏天,暴雨过后,这条路便成为一条“魔鬼小道”,坑塘的水往往会漫过小路,只留下断断续续的残面。几个小伙伴手拉手,打着赤脚,走着走着,就会听见“扑通”落水的声音。好在坑塘边的坡很缓,水也并不深,都能顺利爬上来。如果遇到连绵的阴雨天气,那就糟糕了,村里到处都是泥疗,猪粪、鸡屎被泡得到处流,一些碎石头、碎砖块不知从哪儿钻出来,一不小心就得脚生疼。从家里到学校不过三百米的路,我们一路上不知要踩到多少粪便,看着脚趾缝里挤出来黑色或黄色的污物,散发着臭味的粪便,那浑身的汗毛都要竖起来。虽然如此,小学前的坑塘仍然留有我美好的回忆。坑塘里种满了莲藕,一到夏天,青青的荷叶铺满整个坑塘,间或有粉红色的花高高地冒出来,随风摇曳。然后,慢慢变成莲蓬,里面的莲子圆圆的,鼓鼓的。等不及到成熟的时候,趁大人不注意,我们几个小伙伴会手拉手,连成一串, 进水里,去摘那最近的莲蓬。那莲子,咬一口,满嘴的清香。还有就是那个有着青石桥的坑塘。青石桥把一个大的坑塘分为两个,左边坑塘进入到村庄里面,右边坑塘往外延伸到公路旁,旁边有一条较宽的土路,也从村子的另一边通向公路。土路往上,就是梁家的自留地,每家约有几分地,种些辣椒、茄子、萝卜等蔬菜自给。路和自留地中间有一棵野生的大桑葚树。每到春末夏初,紫红的桑葚结满一树,女生用土块、棍子打,桑葚落了下来,砸进土里,浸满了灰尘,根本无法吃。那些男孩儿却“蹭蹭”地爬上去,摘满一兜,一溜烟就跑。

**示例改写：**
整整下了两天的雨。雨水洗刷下的原野清新、干净,树叶、庄稼都绿得发亮,灰暗的天空形成了一个封闭、安静而又辽阔的世界。而我们就在这时大时小的雨雾中穿行。世界那么小又那么无穷无尽,我们像在孤独中漫游,又像在无限神秘的大地中探索。公路两旁的水沟在多年的干涸之后终于又获得了新生,河流翻出了巨大的波浪和旋涡,在树林间游戏。我仿佛又回到了那湿润的童年,雨后在小沟渠捉鱼,踩水的欢乐和对自然无邪的亲近,每个人在此时都是最纯洁的。雨季来了。虽然不是南方,但每年的这个季节总会有十几天在连续下雨。我喜欢这样的雨天,雨“哗哗”下着,但并不阴暗,灰色的、发亮的天空辽阔、肃穆,给人一种庄严与阔大之感。河坡的树林是近几年才栽种的,林间还没有长出足够覆盖地面的草。赤脚踩在沙土路上,细细的、湿湿的砂石轻 人的脚,微疼微痒,感觉非常舒服。河水“哗哗”奔腾而去,充满力量和向往,那巨大的芦苇丛接受着雨水的冲刷,稳重而又充满生命力。雨中的河,升腾着雾气,苍茫无边,却又具有永恒的清新。河坡地里散落着许多小屋,基本上都是人们为看守庄稼而建的,在一片片空阔的沙地上,种了许多西瓜和花生,它们最适宜在沙地上种植。偶尔可以看到一两个身影在西瓜地里忙碌,估计是在检查西瓜的情况。这样的连阴雨对种瓜的人来说,是非常不好的事情。我们在一个开着门的小屋前张望,里面有一位妇女正在做家务,旁边一个三四岁的女孩子在玩耍。听到我们的声音,那位妇女扭过身来。哥哥笑了起来,这不是焕嫂子吗?

**第一步：详细提取时间线和关键事件**
- 识别所有时间线索和具体事件
- 建立完整的时间顺序
- 标记每个重要事件的具体细节

**第二步：分析人物关系和互动**
- 识别所有提到的人物及其关系
- 记录人物之间的具体互动和对话
- 分析人物情感变化

**第三步：梳理事件逻辑和因果关系**
- 提取所有主要和次要事件
- 分析事件之间的因果关系
- 保留事件的具体情境和背景

**第四步：理解情感基调和细节**
- 分析原始情感表达的细微差别
- 保持情感真实性，不美化负面情绪
- 保留情感表达的具体触发点

**第五步：基于以上详细分析进行改写，确保：**
- 保留原文的具体细节
- 字数不少于原文的20%
- 维持原文的个人化表达风格

**重要要求：**
1. 保持原文的所有重要细节和具体内容，不要过度概括
2. 保留原文的情感和个人化表达
3. 字数不少于原文的20%
4. 按照时间顺序和主题进行组织，但不要丢失具体事例

**现在请改写以下内容：**
{oral_text}
"""

class OralHistoryRewriter:
    def __init__(self):
        self.prompt_templates = {
            'base': BASE_PROMPT,
            'few_shot': FEW_SHOT_PROMPT,
            'cot': COT_PROMPT,
            'combined': COMBINED_PROMPT
        }
    
    @weave.op()
    def rewrite_text(self, method: str, oral_text: str, temperature: float = 0.3) -> str:
        """使用指定方法重写口述文本"""
        prompt = self.prompt_templates[method].format(oral_text=oral_text)
        
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "你是一个专业的口述历史整理专家，擅长保留原始细节和个人化表达。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"API调用错误: {str(e)}")
            return f"Error: {str(e)}"

class LongTextOralHistoryExperiment:
    def __init__(self):
        self.rewriter = OralHistoryRewriter()
    
    def read_docx_file(self, file_path: str) -> str:
        """读取docx文件内容"""
        try:
            from docx import Document
            doc = Document(file_path)
            full_text = []
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    full_text.append(paragraph.text)
            return '\n'.join(full_text)
        except ImportError:
            print("未安装python-docx库，无法读取docx文件")
            print("请运行: pip install python-docx")
            return ""
        except Exception as e:
            print(f"读取docx文件时出错: {str(e)}")
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    return f.read()
            except:
                return ""
    
    def clean_interview_text(self, text: str) -> str:
        """清理采访文本，移除时间戳和说话人标记"""
        # 移除时间戳格式 (00:00:00)
        text = re.sub(r'\(\d{2}:\d{2}:\d{2}\):', '', text)
        # 移除说话人标记
        text = re.sub(r'访谈人:', '', text)
        text = re.sub(r'Jenny:', '', text)
        # 合并多余的空行
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()
    
    def process_with_method(self, long_text: str, method: str, experiment_name: str) -> Dict:
        """使用特定方法处理整个长文本"""
        # 清理文本
        cleaned_text = self.clean_interview_text(long_text)
        print(f"  开始处理整个文本 (原始长度: {len(long_text)} 字符, 清理后: {len(cleaned_text)} 字符)")
        
        # 启动W&B
        wandb.init(
            project="oral-history-long-text-v3",
            name=experiment_name,
            config={
                "method": method,
                "original_length": len(long_text),
                "cleaned_length": len(cleaned_text)
            },
            reinit=True
        )
        
        # 记录开始时间
        start_time = datetime.now()
        
        # 直接处理整个文本
        rewrite = self.rewriter.rewrite_text(method, cleaned_text)
        
        # 计算处理时间
        processing_time = (datetime.now() - start_time).total_seconds()
        
        # 检查是否成功改写
        is_rewritten = (rewrite != cleaned_text and 
                       len(rewrite) > len(cleaned_text) * 0.5 and 
                       not rewrite.startswith("Error:"))
        
        # 创建结果记录
        result = {
            'original_text': long_text,
            'cleaned_text': cleaned_text,
            'rewritten_text': rewrite,
            'original_length': len(long_text),
            'cleaned_length': len(cleaned_text),
            'rewritten_length': len(rewrite),
            'compression_ratio': len(rewrite) / len(cleaned_text) if len(cleaned_text) > 0 else 1,
            'is_rewritten': is_rewritten,
            'processing_time_seconds': processing_time
        }
        
        # 计算统计信息
        stats = {
            "original_length": len(long_text),
            "cleaned_length": len(cleaned_text),
            "rewritten_length": len(rewrite),
            "compression_ratio": len(rewrite) / len(cleaned_text) if len(cleaned_text) > 0 else 1,
            "processing_time_seconds": processing_time,
            "is_rewritten": is_rewritten,
            "detail_preservation_score": min(1.0, len(rewrite) / len(cleaned_text))  # 细节保留评分
        }
        
        # 记录到W&B
        wandb.log(stats)
        wandb.log({
            "original_text_sample": long_text[:800] + "..." if len(long_text) > 800 else long_text,
            "rewritten_text_sample": rewrite[:800] + "..." if len(rewrite) > 800 else rewrite
        })
        
        wandb.finish()
        
        return {
            "full_rewritten_text": rewrite,
            "result": result,
            "statistics": stats
        }
    
    def run_complete_experiment(self, long_text_path: str, experiment_name: str):
        """运行完整的长文本实验"""
        # 读取长文本
        if long_text_path.endswith('.docx'):
            long_text = self.read_docx_file(long_text_path)
        else:
            with open(long_text_path, 'r', encoding='utf-8') as f:
                long_text = f.read()
        
        if not long_text:
            print("无法读取文件内容")
            return {}
        
        print(f"读取文本，总长度: {len(long_text)} 字符")
        
        # 为每种方法运行实验
        all_results = {}
        
        for method in ['base', 'few_shot', 'cot', 'combined']:
            print(f"\n🔬 开始方法: {method}")
            
            method_experiment_name = f"{experiment_name}_{method}_v3"
            result = self.process_with_method(long_text, method, method_experiment_name)
            all_results[method] = result
            
            # 保存结果到文件
            output_filename = f"rewritten_{method}_{experiment_name}_v3.txt"
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(result['full_rewritten_text'])
            
            stats = result['statistics']
            print(f"保存结果到: {output_filename}")
            print(f"处理时间: {stats['processing_time_seconds']:.2f}秒")
            print(f"压缩比例: {stats['compression_ratio']:.2f}")
            print(f"细节保留评分: {stats['detail_preservation_score']:.2f}")
        
        return all_results

def generate_detailed_analysis_report(results: Dict, experiment_name: str):
    """生成详细分析报告"""
    print(f"\n实验 '{experiment_name}' 详细分析报告")
    print("=" * 60)
    
    for method, result in results.items():
        stats = result['statistics']
        print(f"\n方法: {method.upper()}")
        print(f"长度统计:")
        print(f"     原始长度: {stats['original_length']} 字符")
        print(f"     清理后长度: {stats['cleaned_length']} 字符") 
        print(f"     改写后长度: {stats['rewritten_length']} 字符")
        print(f"质量指标:")
        print(f"     压缩比例: {stats['compression_ratio']:.2f}")
        print(f"     细节保留评分: {stats['detail_preservation_score']:.2f}")
        print(f"     是否成功改写: {'是' if stats['is_rewritten'] else '否'}")
        print(f"性能指标:")
        print(f"     处理时间: {stats['processing_time_seconds']:.2f}秒")
        
        # 质量评估
        if stats['compression_ratio'] > 0.8:
            print(f"质量评估: 优秀 - 保留了大部分细节")
        elif stats['compression_ratio'] > 0.6:
            print(f"质量评估: 良好 - 保留了较多细节") 
        elif stats['compression_ratio'] > 0.4:
            print(f"质量评估: 一般 - 部分细节丢失")
        else:
            print(f"质量评估: 较差 - 过度概括")

def run_improved_experiment():
    """运行改进后的实验"""
    print("开始改进版老年人口述史AI改写实验...")
    print("主要改进：")
    print("增强提示词，要求保留更多细节")
    print("增加max_tokens限制")
    print("添加文本清理功能") 
    print("增加细节保留评分")
    print("使用更详细的few-shot示例")
    
    # 初始化实验
    experiment = LongTextOralHistoryExperiment()
    
    # 运行实验
    long_text_path = "Jenny-老年人访谈-转写.docx"
    experiment_name = "老年人采访改进实验"
    
    results = experiment.run_complete_experiment(long_text_path, experiment_name)
    
    # 生成详细分析报告
    generate_detailed_analysis_report(results, experiment_name)
    
    # 保存比较分析
    save_comparison_analysis(results, experiment_name)
    
    return results

def save_comparison_analysis(results: Dict, experiment_name: str):
    """保存比较分析结果"""
    comparison_data = []
    for method, result in results.items():
        stats = result['statistics']
        comparison_data.append({
            '方法': method,
            '原始长度': stats['original_length'],
            '改写长度': stats['rewritten_length'], 
            '压缩比例': f"{stats['compression_ratio']:.2f}",
            '细节保留评分': f"{stats['detail_preservation_score']:.2f}",
            '处理时间(秒)': f"{stats['processing_time_seconds']:.2f}",
            '成功改写': '是' if stats['is_rewritten'] else '否'
        })
    
    df = pd.DataFrame(comparison_data)
    df.to_csv(f"experiment_comparison_{experiment_name}.csv", index=False, encoding='utf-8-sig')
    print(f"\n比较分析已保存至: experiment_comparison_{experiment_name}.csv")
    print(df.to_string(index=False))

if __name__ == "__main__":
    run_improved_experiment()

c:\Users\yuanzhouyan\AppData\Local\Programs\Python\Python313\Lib\site-packages\rich\live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


weave: Logged in as Weights & Biases user: yuan30jancis.
weave: View Weave data at https://wandb.ai/yuan30jancis-fudan-university-school-of-management/oral-history-rewriting-experiment-v3/weave


🎯 开始改进版老年人口述史AI改写实验...
主要改进：
✅ 增强提示词，要求保留更多细节
✅ 增加max_tokens限制
✅ 添加文本清理功能
✅ 增加细节保留评分
✅ 使用更详细的few-shot示例
📖 读取文本，总长度: 18790 字符

🔬 开始方法: base
  开始处理整个文本 (原始长度: 18790 字符, 清理后: 15985 字符)


wandb: Initializing weave.


...

wandb: Initializing weave.


c:\Users\yuanzhouyan\AppData\Local\Programs\Python\Python313\Lib\site-packages\rich\live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

weave: Logged in as Weights & Biases user: yuan30jancis.
weave: View Weave data at https://wandb.ai/yuan30jancis-fudan-university-school-of-management/oral-history-long-text-v3/weave
weave: 🍩 https://wandb.ai/yuan30jancis-fudan-university-school-of-management/oral-history-long-text-v3/r/call/019a66e3-4025-7065-9168-3c41ac261ae4
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


cleaned_length,▁
compression_ratio,▁
detail_preservation_score,▁
original_length,▁
processing_time_seconds,▁
rewritten_length,▁
cleaned_length,15985
compression_ratio,0.12305
detail_preservation_score,0.12305
is_rewritten,False
original_length,18790


💾 保存结果到: rewritten_combined_老年人采访改进实验_v3.txt
⏱️  处理时间: 50.90秒
📊 压缩比例: 0.12
🔍 细节保留评分: 0.12

📊 实验 '老年人采访改进实验' 详细分析报告

方法: BASE
  📏 长度统计:
     原始长度: 18790 字符
     清理后长度: 15985 字符
     改写后长度: 1263 字符
  📈 质量指标:
     压缩比例: 0.08
     细节保留评分: 0.08
     是否成功改写: ❌ 否
  ⏱️  性能指标:
     处理时间: 34.35秒
  🎯 质量评估: ❌ 较差 - 过度概括

方法: FEW_SHOT
  📏 长度统计:
     原始长度: 18790 字符
     清理后长度: 15985 字符
     改写后长度: 1683 字符
  📈 质量指标:
     压缩比例: 0.11
     细节保留评分: 0.11
     是否成功改写: ❌ 否
  ⏱️  性能指标:
     处理时间: 45.11秒
  🎯 质量评估: ❌ 较差 - 过度概括

方法: COT
  📏 长度统计:
     原始长度: 18790 字符
     清理后长度: 15985 字符
     改写后长度: 1762 字符
  📈 质量指标:
     压缩比例: 0.11
     细节保留评分: 0.11
     是否成功改写: ❌ 否
  ⏱️  性能指标:
     处理时间: 45.31秒
  🎯 质量评估: ❌ 较差 - 过度概括

方法: COMBINED
  📏 长度统计:
     原始长度: 18790 字符
     清理后长度: 15985 字符
     改写后长度: 1967 字符
  📈 质量指标:
     压缩比例: 0.12
     细节保留评分: 0.12
     是否成功改写: ❌ 否
  ⏱️  性能指标:
     处理时间: 50.90秒
  🎯 质量评估: ❌ 较差 - 过度概括

📋 比较分析已保存至: experiment_comparison_老年人采访改进实验.csv
      方法  原始长度  改写长度 压缩比例 细节保留评分 处理时间(秒) 成功改写
    bas